In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

**Plan of action:**<br>
- **DONE**Figure out which columns are needed in the stats data: pos,player,team,pass_yds,pass_td,pass_int,rec_yds,rec_tds,rush_yds,rush_td,surface,game_date 
- **DONE**Create a list of the injured players and make new df to merge that with the stats.
- **DONE**Lump all three seasons to get averages or break it up for each season (Went with lump)
- Merge the offense/defense/kicking df's to line up the player's 
- **DONE**Make list of injured players in each year (cutoff at end of 2020 season). WENT WITH BEFORE/AFTER STATS
- From that list, compare the average of all the player's stats to the injured ones.
- Maybe only focus on the offense stats?
- Find the percent of players by position who get injured.
- **DONE**CONCAT ALL OF THE INJURY DF'S TOGETHER (ADD A COLUMN TO SPECIFY THE INJURY FOR THE FILTERING)
- Make a percentage of after yards in Tableau for bar chart.
- Make concussions a separate thing.
- See if there's correlation between injuries and player positions. Make graph that shows the percent breakdown by pos.
- Make slideshow for showing data/process of what I did then show dashboard.

### Pull in the data (injury reports/player stats)

In [ ]:
acl_report = pd.read_csv('../data/acl_report.csv', header = 1)

In [ ]:
concussion_report = pd.read_csv('../data/concussion_report.csv', header = 1)

In [ ]:
shoulder_report = pd.read_csv('../data/shoulder_report.csv', header = 1)

In [ ]:
ankle_report = pd.read_csv('../data/ankle_report.csv', header = 1)

In [ ]:
offense_stats = pd.read_csv('../data/nfl_offense.csv')

In [ ]:
defense_stats = pd.read_csv('../data/nfl_defense.csv')

In [ ]:
kicking_stats = pd.read_csv('../data/nfl_kicking.csv')

### Data Cleaning

In [ ]:
acl_report = acl_report[['Date', 'Team', 'Relinquished', 'Notes']]
concussion_report = concussion_report[['Date', 'Team', 'Relinquished', 'Notes']]
shoulder_report = shoulder_report[['Date', 'Team', 'Relinquished', 'Notes']]
ankle_report = ankle_report[['Date', 'Team', 'Relinquished', 'Notes']]

In [ ]:
acl_report = acl_report.rename(columns = {'Relinquished':'Player'})
concussion_report = concussion_report.rename(columns = {'Relinquished':'Player'})
shoulder_report = shoulder_report.rename(columns = {'Relinquished':'Player'})
ankle_report = ankle_report.rename(columns = {'Relinquished':'Player'})

In [ ]:
acl_report['Player'] = acl_report['Player'].str[2:]
concussion_report['Player'] = concussion_report['Player'].str[2:]
shoulder_report['Player'] = shoulder_report['Player'].str[2:]
ankle_report['Player'] = ankle_report['Player'].str[2:]

In [ ]:
acl_report

In [ ]:
offense_stats

In [ ]:
#offense_stats = offense_stats[['player','team','pass_yds','pass_td','pass_int','rec_yds','rec_td','rush_yds','rush_td','Surface','game_date']]

In [ ]:
offense_stats

In [ ]:
offense_grouped = offense_stats.groupby(['player','pos'])[['pass_yds','pass_td','pass_int','rec_yds','rec_td','rush_yds','rush_td']].sum()

In [ ]:
offense_grouped = offense_grouped.reset_index()

In [ ]:
offense_grouped

In [ ]:
offense_grouped.loc[offense_grouped['pass_yds'] > 3000]['pass_yds'].mean()/48

In [ ]:
offense_grouped.loc[offense_grouped['rec_yds'] >= 300]['rec_yds'].mean()/3

In [ ]:
acl_report.Date.unique()

In [ ]:
acl_report = acl_report[acl_report['Date'] != 'Date']
shoulder_report = shoulder_report[shoulder_report['Date'] != 'Date']
concussion_report = concussion_report[concussion_report['Date'] != 'Date']
ankle_report = ankle_report[ankle_report['Date'] != 'Date']

In [ ]:
acl_report['Date'] = pd.to_datetime(acl_report['Date'], format = '%Y-%m-%d')
shoulder_report['Date'] = pd.to_datetime(shoulder_report['Date'], format = '%Y-%m-%d')
concussion_report['Date'] = pd.to_datetime(concussion_report['Date'], format = '%Y-%m-%d')
ankle_report['Date'] = pd.to_datetime(ankle_report['Date'], format = '%Y-%m-%d')

In [ ]:
acl_report

In [ ]:
acl_report.Player.value_counts()

In [ ]:
acl_players = acl_report.drop_duplicates(subset = 'Player', keep = 'first')
shoulder_players = shoulder_report.drop_duplicates(subset = 'Player', keep = 'first')
concussion_players = concussion_report.drop_duplicates(subset = 'Player', keep = 'first')
ankle_players = ankle_report.drop_duplicates(subset = 'Player', keep = 'first')

In [ ]:
acl_offense = pd.merge(acl_players, offense_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
shoulder_offense = pd.merge(shoulder_players, offense_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
concussion_offense = pd.merge(concussion_players, offense_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
ankle_offense = pd.merge(ankle_players, offense_grouped, how = 'inner', left_on = 'Player', right_on = 'player')

In [ ]:
acl_offense.sort_values(by = 'pass_yds', ascending = False)

In [ ]:
acl_offense.pos.value_counts()

In [ ]:
shoulder_offense.pos.value_counts()

In [ ]:
concussion_offense.pos.value_counts()

In [ ]:
ankle_offense.pos.value_counts()

In [ ]:
ankle_offense

In [ ]:
ankle_offense.info()

In [ ]:
offense_stats['game_date'] = pd.to_datetime(offense_stats['game_date'], format = '%Y-%m-%d')

In [ ]:
offense_stats.info()

In [ ]:
for ind,row in offense_stats.iterrows():
    if row.game_date > pd.to_datetime('2022-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2022 Season'
    elif row.game_date > pd.to_datetime('2021-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2021 Season'
    elif row.game_date > pd.to_datetime('2020-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2020 Season'
    elif row.game_date > pd.to_datetime('2019-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2019 Season'
    else: print('')

In [ ]:
offense_stats

In [ ]:
offense_season_grouped = offense_stats.groupby(['player','pos', 'season'])[['pass_yds','pass_td','pass_int','rec_yds','rec_td','rush_yds','rush_td']].sum()

In [ ]:
offense_season_grouped = offense_season_grouped.reset_index()

In [ ]:
offense_season_grouped

In [ ]:
acl_offense_season = pd.merge(acl_players, offense_season_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
shoulder_offense_season = pd.merge(shoulder_players, offense_season_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
concussion_offense_season = pd.merge(concussion_players, offense_season_grouped, how = 'inner', left_on = 'Player', right_on = 'player')
ankle_offense_season = pd.merge(ankle_players, offense_season_grouped, how = 'inner', left_on = 'Player', right_on = 'player')

In [ ]:
concussion_offense_season

In [ ]:
#offense_season_grouped.to_csv('../data/offense_season_grouped.csv')
#acl_offense_season.to_csv('../data/acl_offense_season.csv')

In [ ]:
acl_offense_season.loc[acl_offense_season['pos'] == 'TE']

for ind,row in acl_offense_season.iterrows():
    if row.season == '2022 Season':
        offense_stats.loc[ind, 'season'] = pd.to_datetime('2022-9-1', format = '%Y-%m-%d')
    elif row.game_date > pd.to_datetime('2021-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2021 Season'
    elif row.game_date > pd.to_datetime('2020-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2020 Season'
    elif row.game_date > pd.to_datetime('2019-9-1', format = '%Y-%m-%d'):
        offense_stats.loc[ind, 'season'] = '2019 Season'
    else: print('')

In [ ]:
for ind,row in acl_offense_season.iterrows():
    if row.season == '2022 Season':
        acl_offense_season.loc[ind, 'season_start'] = pd.to_datetime('2022-9-1', format = '%Y-%m-%d')
    if row.season == '2021 Season':
        acl_offense_season.loc[ind, 'season_start'] = pd.to_datetime('2021-9-1', format = '%Y-%m-%d')
    if row.season == '2020 Season':
        acl_offense_season.loc[ind, 'season_start'] = pd.to_datetime('2020-9-1', format = '%Y-%m-%d')
    if row.season == '2019 Season':
        acl_offense_season.loc[ind, 'season_start'] = pd.to_datetime('2019-9-1', format = '%Y-%m-%d')

In [ ]:
for ind,row in acl_offense_season.iterrows():
    if row.season_start > row.Date:
        acl_offense_season.loc[ind, 'after_injury'] = 'after'
    elif row.season_start < row.Date:
        acl_offense_season.loc[ind, 'before_injury'] = 'before'
    elif row.season_start < row.Date:
        acl_offense_season.loc[ind, 'after_injury'] = ''
    elif row.season_start > row.Date:
        acl_offense_season.loc[ind, 'before_injury'] = ''

In [ ]:
acl_offense_season['total_yds'] = acl_offense_season['pass_yds'] + acl_offense_season['rec_yds'] + acl_offense_season['rush_yds']

In [ ]:
acl_offense_season = acl_offense_season.fillna('')

In [ ]:
acl_offense_season.loc[acl_offense_season['Player'] == 'Chris Godwin']

In [ ]:
acl_offense_before = acl_offense_season.loc[acl_offense_season['before_injury'] == 'before']
acl_offense_after = acl_offense_season.loc[acl_offense_season['after_injury'] == 'after']

In [ ]:
#acl_offense_before.to_csv('../data/acl_offense_before.csv')
#acl_offense_after.to_csv('../data/acl_offense_after.csv')

In [ ]:
#acl_offense_season.to_csv('../data/acl_offense_season.csv')

In [ ]:
acl_offense_season

In [ ]:
for ind,row in acl_offense_season.iterrows():
    if row.season_start > row.Date:
        acl_offense_season.loc[ind, 'before_after_injury'] = 'after'
    elif row.season_start < row.Date:
        acl_offense_season.loc[ind, 'before_after_injury'] = 'before'
    else: acl_offense_season.loc[ind, 'before_after_injury'] = ''

In [ ]:
acl_offense_season

In [ ]:
acl_offense_after

In [ ]:
acl_offense_season['injury'] = 'acl'

In [ ]:
shoulder_offense_season['injury'] = 'shoulder'
concussion_offense_season['injury'] = 'concussion'
ankle_offense_season['injury'] = 'ankle'

In [ ]:
acl_offense_season = acl_offense_season.drop(columns = ['Player','Notes','after_injury','before_injury','before_after_injury'])

In [ ]:
injury_merge = pd.concat([shoulder_offense_season, ankle_offense_season])

In [ ]:
injury_merge = injury_merge.drop(columns = ['Player', 'Notes'])

In [ ]:
injury_merge.info()

In [ ]:
acl_offense_season.info()

In [ ]:
acl_off = acl_offense_season.drop(columns = ['season_start', 'total_yds'])

In [ ]:
injury_offense = pd.concat([acl_off, injury_merge])

In [ ]:
injury_offense

In [ ]:
injury_offense['total_yds'] = injury_offense['pass_yds'] + injury_offense['rec_yds'] + injury_offense['rush_yds']

In [ ]:
injury_offense

In [ ]:
for ind,row in injury_offense.iterrows():
    if row.season == '2022 Season':
        injury_offense.loc[ind, 'season_start'] = pd.to_datetime('2022-9-1', format = '%Y-%m-%d')
    if row.season == '2021 Season':
        injury_offense.loc[ind, 'season_start'] = pd.to_datetime('2021-9-1', format = '%Y-%m-%d')
    if row.season == '2020 Season':
        injury_offense.loc[ind, 'season_start'] = pd.to_datetime('2020-9-1', format = '%Y-%m-%d')
    if row.season == '2019 Season':
        injury_offense.loc[ind, 'season_start'] = pd.to_datetime('2019-9-1', format = '%Y-%m-%d')

In [ ]:
injury_offense

In [ ]:
for ind,row in injury_offense.iterrows():
    if row.season_start > row.Date:
        injury_offense.loc[ind, 'before_after_injury'] = 'after'
    elif row.season_start < row.Date:
        injury_offense.loc[ind, 'before_after_injury'] = 'before'
    else: injury_offense.loc[ind, 'before_after_injury'] = ''

In [ ]:
injury_offense = injury_offense.rename(columns = {'Date':'injury_date'})

In [ ]:
injury_offense.info()

In [ ]:
injury_offense = injury_offense.iloc[:, [0,1,2,3,4,14,5,6,7,8,9,10,11,13,15,12]]

In [ ]:
#injury_offense.to_csv('../data/injury_offense.csv')

In [ ]:
injury_offense.loc[injury_offense['player'] == 'Jimmy Garoppolo']

In [ ]:
concussion_offense_season.sort_values(by = 'player')

In [ ]:
concussion_stats = concussion_offense_season.groupby(['Date', 'player', 'pos', 'injury'])[['pass_yds','rec_yds','rush_yds']].sum().reset_index()

In [ ]:
concussion_stats.player.value_counts()

In [ ]:
concussion_stats.loc[concussion_stats['player'] == 'Taysom Hill']

In [ ]:
concussion_report

In [ ]:
for ind,row in concussion_report.iterrows():
    if row.Date > pd.to_datetime('2022-9-1', format = '%Y-%m-%d'):
        concussion_report.loc[ind, 'season'] = '2022 Season'
    elif row.Date > pd.to_datetime('2021-9-1', format = '%Y-%m-%d'):
        concussion_report.loc[ind, 'season'] = '2021 Season'
    elif row.Date > pd.to_datetime('2020-9-1', format = '%Y-%m-%d'):
        concussion_report.loc[ind, 'season'] = '2020 Season'
    elif row.Date > pd.to_datetime('2019-9-1', format = '%Y-%m-%d'):
        concussion_report.loc[ind, 'season'] = '2019 Season'
    elif row.Date >= pd.to_datetime('2018-7-1', format = '%Y-%m-%d'):
        concussion_report.loc[ind, 'season'] = '2018 Season'

In [ ]:
concussion_report

In [ ]:
concussion_report.Player.value_counts()

In [ ]:
concussion_report.loc[concussion_report['Player'] == 'Buster Skrine']

In [ ]:
concussion_offense.player.value_counts()

In [ ]:
injury_offense = injury_offense.loc[injury_offense['season'] != '2022 Season']

In [ ]:
injury_offense

In [ ]:
offense_season_grouped = offense_season_grouped.loc[offense_season_grouped['season'] != '2022 Season']

In [ ]:
for ind,row in offense_season_grouped.iterrows():
    if row.pos == 'HB':
        offense_season_grouped['pos'] = offense_season_grouped['pos'].replace('HB', 'RB')
    if row.pos == 'WR/R':
        offense_season_grouped['pos'] = offense_season_grouped['pos'].replace('WR/R', 'WR')

In [ ]:
offense_season_grouped